In [1]:
import scipy.io
import scipy
import numpy
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
import matplotlib.pyplot as plt
import pandas
from keras.layers import Dropout
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from jupyterthemes import jtplot
jtplot.style()

#read in training data

#read in test data

# Define the scaler
scaler = StandardScaler().fit(TrainSig)

# Scale the train set
TrainSig = scaler.transform(TrainSig)

# Scale the test set
TestSig = scaler.transform(TestSig)

Using Theano backend.


NameError: name 'TrainSig' is not defined

In [ ]:
# create model
numpy.random.seed(2)
model = Sequential()
hidden_units = 300
dropout_rate = 0.45
e = 60

# model.add(Dropout(0.1,input_shape=(TrainSig.shape[1],)))
model.add(Dense(input_dim=TrainSig.shape[1],
                kernel_initializer='he',
                activation='relu',
                units=hidden_units))
model.add(Dropout(dropout_rate))
model.add(Dense(input_dim=hidden_units,
                kernel_initializer='normal',
                activation='relu',
                units=hidden_units))
model.add(Dropout(dropout_rate))
model.add(Dense(input_dim=hidden_units,
                kernel_initializer='normal',
                units=TrainParams.shape[1]))
# Compile model
model.compile(loss='mean_squared_error', optimizer='Adam')
history = model.fit(TrainSig,
         TrainParams,
         epochs=e,
         batch_size=50,
         verbose=0,
         validation_split=0.1)


# predict test values using NN
TestPredict = model.predict(TestSig)


# summarize history for accuracy
plt.plot(history.history['loss'])
plt.title('Epochs vs Validation Loss')
plt.ylabel('Mean squared error')
plt.xlabel('Epoch')
plt.legend(['val_loss'], loc='upper left')
plt.show()

